# cuPauliProp Truncation Strategies - 10-Qubit 예제

## 개요

**목표**: Truncation 전략을 사용한 Pauli propagation 메모리 최적화

**핵심 전략**:
- **Coefficient-based truncation**: |coef| < 1e-4인 항 제거
- **Pauli weight-based truncation**: non-identity Pauli 개수 > 8인 항 제거
- **주기적 적용**: 10개 gate마다 truncation 실행

**회로 구조**:
- 10-qubit 시스템
- 4 layers
- 각 layer: 3×rotations(RX, RY, RZ) + CNOT gates

---

## 참고 문서
- [cuPauliProp Kicked Ising Example](https://docs.nvidia.com/cuda/cuquantum/latest/cupauliprop/examples.html)
- C++ 예제 기반 Python ctypes 구현

---
## Section 1: Import Libraries and Setup

In [1]:
import numpy as np
import cupy as cp
import ctypes
from cuquantum.bindings import cupauliprop as cupp
import time

print("✓ Libraries loaded successfully")
print(f"  NumPy: {np.__version__}")
print(f"  CuPy: {cp.__version__}")

✓ Libraries loaded successfully
  NumPy: 2.4.1
  CuPy: 13.6.0


In [2]:
# Create cuPauliProp handle
handle = cupp.create()
print("✓ cuPauliProp handle created")

✓ cuPauliProp handle created


---
## Section 2: System Parameters and Observable Initialization

In [3]:
num_qubits = 10
num_packed_ints = cupp.get_num_packed_integers(num_qubits)

print(f"System Configuration:")
print(f"  Qubits: {num_qubits}")
print(f"  Packed integers per Pauli: {num_packed_ints}")

# Observable: Z_0 (coefficient = 1.0)
d_input_pauli = cp.array([0b0, 0b100000000], dtype=cp.uint64)  # Z_0: X_mask=0, Z_mask=1
d_input_coef = cp.array([1.0], dtype=cp.float64)

print(f"\n✓ Observable defined: Z_0")
print(f"  X mask: {int(d_input_pauli[0]):010b}")
print(f"  Z mask: {int(d_input_pauli[1]):010b}")
print(f"  Coefficient: {float(d_input_coef[0])}")

System Configuration:
  Qubits: 10
  Packed integers per Pauli: 1



✓ Observable defined: Z_0
  X mask: 0000000000
  Z mask: 0100000000
  Coefficient: 1.0


---
## Section 3: GPU Memory and Workspace Allocation

In [4]:
max_terms = 500000  # 충분한 버퍼 (4 layers × 30 gates per layer with branching)

pauli_buffer_size = 2 * num_packed_ints * max_terms * 8  # bytes
coef_buffer_size = max_terms * 8  # bytes (float64)

# Allocate GPU buffers
d_input_pauli_buffer = cp.zeros(2 * num_packed_ints * max_terms, dtype=cp.uint64)
d_input_coef_buffer = cp.zeros(max_terms, dtype=cp.float64)
d_output_pauli_buffer = cp.zeros(2 * num_packed_ints * max_terms, dtype=cp.uint64)
d_output_coef_buffer = cp.zeros(max_terms, dtype=cp.float64)

# Initialize with observable
d_input_pauli_buffer[:len(d_input_pauli)] = d_input_pauli
d_input_coef_buffer[:len(d_input_coef)] = d_input_coef

print(f"Memory Allocation:")
print(f"  max_terms: {max_terms}")
print(f"  Pauli buffer: {pauli_buffer_size / 1024**2:.1f} MB × 2")
print(f"  Coef buffer: {coef_buffer_size / 1024**2:.1f} MB × 2")
print(f"  Total expansion buffers: {2 * (pauli_buffer_size + coef_buffer_size) / 1024**2:.1f} MB")

# Helper function to reset expansions
def create_expansions():
    """Create fresh input/output Pauli expansions"""
    input_exp = cupp.create_pauli_expansion(
        handle, num_qubits,
        d_input_pauli_buffer.data.ptr, pauli_buffer_size,
        d_input_coef_buffer.data.ptr, coef_buffer_size,
        1,  # dataType: CUDA_R_64F
        len(d_input_coef),  # numTerms
        1,  # isSorted
        1   # isUnique
    )
    output_exp = cupp.create_pauli_expansion(
        handle, num_qubits,
        d_output_pauli_buffer.data.ptr, pauli_buffer_size,
        d_output_coef_buffer.data.ptr, coef_buffer_size,
        1, 0, 0, 0  # empty
    )
    return input_exp, output_exp

input_expansion, output_expansion = create_expansions()
print(f"\n✓ Pauli expansions created")

Memory Allocation:
  max_terms: 500000
  Pauli buffer: 7.6 MB × 2
  Coef buffer: 3.8 MB × 2
  Total expansion buffers: 22.9 MB

✓ Pauli expansions created


In [5]:
# Allocate workspace
buffer_mem = (d_input_pauli_buffer.nbytes + d_input_coef_buffer.nbytes + 
              d_output_pauli_buffer.nbytes + d_output_coef_buffer.nbytes)
available_mem = cp.cuda.Device().mem_info[0]
workspace_size = int((available_mem - buffer_mem) * 0.6)

print(f"Workspace Allocation:")
print(f"  Available GPU mem: {available_mem / 1024**3:.2f} GB")
print(f"  Buffer mem: {buffer_mem / 1024**2:.1f} MB")
print(f"  Workspace size: {workspace_size / 1024**2:.1f} MB")

d_workspace = cp.cuda.alloc(workspace_size)

workspace = cupp.create_workspace_descriptor(handle)
cupp.workspace_set_memory(
    handle, workspace,
    0,  # CUPAULIPROP_MEMSPACE_DEVICE
    0,  # CUPAULIPROP_WORKSPACE_SCRATCH
    d_workspace.ptr, workspace_size
)
print(f"✓ Workspace allocated")

Workspace Allocation:
  Available GPU mem: 14.45 GB
  Buffer mem: 22.9 MB
  Workspace size: 8864.8 MB
✓ Workspace allocated


---
## Section 4: Circuit Construction with Layer Structure

In [7]:
NUM_LAYERS = 4
theta = np.pi / 4

# Circuit configuration
gate_ops = []
PI = np.pi

print(f"Circuit Configuration:")
print(f"  Layers: {NUM_LAYERS}")
print(f"  Rotation angle: π/4")
print(f"  Qubits: {num_qubits}")

for layer in range(NUM_LAYERS):
    # RX, RY, RZ on each qubit
    for qubit in range(num_qubits):
        rx = cupp.create_pauli_rotation_gate_operator(handle, theta, 1, [qubit], [1])  # X
        gate_ops.append(rx)
        ry = cupp.create_pauli_rotation_gate_operator(handle, theta, 1, [qubit], [3])  # Y
        gate_ops.append(ry)
        rz = cupp.create_pauli_rotation_gate_operator(handle, theta, 1, [qubit], [2])  # Z
        gate_ops.append(rz)

    # CNOT between adjacent qubits
    for qubit in range(num_qubits - 1):
        cnot = cupp.create_clifford_gate_operator(handle, 7, [qubit+1, qubit])  # CNOT
        gate_ops.append(cnot)

print(f"\n✓ Circuit constructed:")
print(f"  Total gates: {len(gate_ops)}")
print(f"  Gates per layer: {len(gate_ops) // NUM_LAYERS}")

Circuit Configuration:
  Layers: 4
  Rotation angle: π/4
  Qubits: 10

✓ Circuit constructed:
  Total gates: 156
  Gates per layer: 39


---
## Section 5: Define Truncation Structures (ctypes)

In [ ]:
# ========================================
# Truncation Strategy Structures (bindings classes)
# ========================================
# Use cuPauliProp's native Python wrappers instead of ctypes/NumPy records

# Truncation strategy constants (match cupaulipropTruncationStrategy_t)
CUPAULIPROP_TRUNCATION_STRATEGY_COEFFICIENT_BASED = 0
CUPAULIPROP_TRUNCATION_STRATEGY_PAULI_WEIGHT_BASED = 1

# Create truncation parameter objects from bindings
coef_trunc_params = cupp.CoefficientTruncationParams()
coef_trunc_params.cutoff = 8e-4

weight_trunc_params = cupp.PauliWeightTruncationParams()
weight_trunc_params.cutoff = 10

print("✓ Truncation param objects created (bindings)")
print(f"  Coefficient cutoff: {coef_trunc_params.cutoff}")
print(f"  Pauli weight cutoff: {weight_trunc_params.cutoff}")

# Build TruncationStrategy objects list (sequence accepted by the binding)
trunc_strat_coef = cupp.TruncationStrategy()
trunc_strat_coef.strategy = CUPAULIPROP_TRUNCATION_STRATEGY_COEFFICIENT_BASED
trunc_strat_coef.param_struct = coef_trunc_params.ptr

trunc_strat_weight = cupp.TruncationStrategy()
trunc_strat_weight.strategy = CUPAULIPROP_TRUNCATION_STRATEGY_PAULI_WEIGHT_BASED
trunc_strat_weight.param_struct = weight_trunc_params.ptr

trunc_strategies = [trunc_strat_coef, trunc_strat_weight]
num_trunc_strategies = len(trunc_strategies)

# Truncation cadence
num_gates_between_truncations = 3

print("✓ Truncation strategies prepared (bindings objects)")
print(f"  Num strategies: {num_trunc_strategies}")
print(f"  Truncation every: {num_gates_between_truncations} gates")

✓ Truncation param objects created (bindings)
  Coefficient cutoff: 0.0008
  Pauli weight cutoff: 10
✓ Truncation strategies prepared (bindings objects)
  Num strategies: 2
  Truncation every: 3 gates


---
## Section 6: Pauli Propagation WITH Truncation (Dual Strategy)

In [9]:
print("="*70)
print("PROPAGATION WITH DUAL TRUNCATION (Coef + Weight)")
print("="*70)

# Reset buffers and expansions
d_input_pauli_buffer.fill(0)
d_input_coef_buffer.fill(0)
d_output_pauli_buffer.fill(0)
d_output_coef_buffer.fill(0)
d_input_pauli_buffer[:len(d_input_pauli)] = d_input_pauli
d_input_coef_buffer[:len(d_input_coef)] = d_input_coef

# Recreate expansions
cupp.destroy_pauli_expansion(input_expansion)
cupp.destroy_pauli_expansion(output_expansion)
input_expansion, output_expansion = create_expansions()

current_input = input_expansion
current_output = output_expansion

start_time = time.time()
term_history_with_trunc = []
max_terms_with_trunc = 0

print(f"\nApplying {len(gate_ops)} gates in reverse order with periodic truncation...\n")

for gate_idx, gate in enumerate(reversed(gate_ops)):
    gate_number = len(gate_ops) - gate_idx

    # Get current term count
    num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    term_history_with_trunc.append(num_terms)
    max_terms_with_trunc = max(max_terms_with_trunc, num_terms)

    # Decide whether to apply truncation
    apply_truncation = (gate_idx % num_gates_between_truncations == 0)
    num_strats = num_trunc_strategies if apply_truncation else 0
    # Pass Python sequence of TruncationStrategy objects
    strats_obj = trunc_strategies if apply_truncation else None

    # Create view
    input_view = cupp.pauli_expansion_get_contiguous_range(
        handle, current_input, 0, num_terms
    )

    # Apply gate with truncation
    cupp.pauli_expansion_view_compute_operator_application(
        handle,
        input_view,
        current_output,
        gate,
        1,      # adjoint=1
        1,      # makeSorted=True
        0,      # keepDuplicates=False
        num_strats,     # num_truncation_strategies
        strats_obj,     # truncation_strategies (sequence or None)
        workspace
    )

    # Destroy view
    cupp.destroy_pauli_expansion_view(input_view)

    # Swap
    current_input, current_output = current_output, current_input

    new_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)

    if gate_idx % 10 == 0 or gate_idx == len(gate_ops) - 1:
        status = "TRUNCATED" if apply_truncation else "no truncation"
        print(f"Gate {gate_number:3d}: {num_terms:7d} → {new_num_terms:7d} terms [{status}]  (num_strats={num_strats})")

end_time = time.time()
final_num_terms_with_trunc = cupp.pauli_expansion_get_num_terms(handle, current_input)

print(f"\n{'='*70}")
print(f"WITH TRUNCATION Results:")
print(f"  Final terms: {final_num_terms_with_trunc}")
print(f"  Max terms during evolution: {max_terms_with_trunc}")
print(f"  Runtime: {end_time - start_time:.2f} sec")
print(f"{'='*70}\n")

PROPAGATION WITH DUAL TRUNCATION (Coef + Weight)

Applying 156 gates in reverse order with periodic truncation...

Gate 156:       1 →       1 terms [TRUNCATED]  (num_strats=2)
Gate 146:       1 →       1 terms [no truncation]  (num_strats=0)
Gate 136:      18 →      27 terms [no truncation]  (num_strats=0)
Gate 126:     729 →     729 terms [TRUNCATED]  (num_strats=2)
Gate 116:   19683 →   19683 terms [no truncation]  (num_strats=0)
Gate 106:   45927 →   45927 terms [no truncation]  (num_strats=0)
Gate  96:  145511 →  124315 terms [TRUNCATED]  (num_strats=2)
Gate  86:  173049 →  217854 terms [no truncation]  (num_strats=0)
Gate  76:  166697 →  166697 terms [no truncation]  (num_strats=0)
Gate  66:  279810 →  184499 terms [TRUNCATED]  (num_strats=2)
Gate  56:  153945 →  205720 terms [no truncation]  (num_strats=0)
Gate  46:  183551 →  219905 terms [no truncation]  (num_strats=0)
Gate  36:  120169 →  120169 terms [TRUNCATED]  (num_strats=2)
Gate  26:  118225 →  160241 terms [no truncatio

In [10]:
# Compute expectation value with truncation
final_view_trunc = cupp.pauli_expansion_get_contiguous_range(
    handle, current_input, 0, final_num_terms_with_trunc
)

result_with_trunc = np.array([0.0], dtype=np.float64)
cupp.pauli_expansion_view_compute_trace_with_zero_state(
    handle,
    final_view_trunc,
    result_with_trunc.ctypes.data,
    workspace
)

expectation_with_trunc = result_with_trunc[0]
print(f"Expectation value (WITH truncation): {expectation_with_trunc:.8f}")

# Store final state for debugging
if current_input == input_expansion:
    buffer_pauli_trunc = d_input_pauli_buffer
    buffer_coef_trunc = d_input_coef_buffer
else:
    buffer_pauli_trunc = d_output_pauli_buffer
    buffer_coef_trunc = d_output_coef_buffer

pauli_data_trunc = buffer_pauli_trunc[:final_num_terms_with_trunc*2*num_packed_ints].get()
coef_data_trunc = buffer_coef_trunc[:final_num_terms_with_trunc].get()

print(f"\nFinal Pauli Expansion (first 10 terms):")
for j in range(min(10, final_num_terms_with_trunc)):
    x_mask = pauli_data_trunc[2*j]
    z_mask = pauli_data_trunc[2*j + 1]
    coef = coef_data_trunc[j]
    print(f"  Term {j}: X={x_mask:010b} Z={z_mask:010b} coef={coef:+.6e}")

Expectation value (WITH truncation): 0.02130400

Final Pauli Expansion (first 10 terms):
  Term 0: X=0000000001 Z=0000000000 coef=+1.897082e-03
  Term 1: X=0000000010 Z=0000000000 coef=+4.038935e-03
  Term 2: X=0000000011 Z=0000000000 coef=+8.949103e-04
  Term 3: X=0000000100 Z=0000000000 coef=+2.173781e-03
  Term 4: X=0000000101 Z=0000000000 coef=-2.350054e-03
  Term 5: X=0000000110 Z=0000000000 coef=+2.525917e-03
  Term 6: X=0000000111 Z=0000000000 coef=-6.955703e-04
  Term 7: X=0000001101 Z=0000000000 coef=+1.183775e-03
  Term 8: X=0000010001 Z=0000000000 coef=+6.935295e-04
  Term 9: X=0000011011 Z=0000000000 coef=+6.170601e-04


---
## Section 7: Pauli Propagation WITHOUT Truncation (Reference)

In [11]:
print("="*70)
print("PROPAGATION WITHOUT TRUNCATION (Reference/Baseline)")
print("="*70)

# Reset buffers and expansions
d_input_pauli_buffer.fill(0)
d_input_coef_buffer.fill(0)
d_output_pauli_buffer.fill(0)
d_output_coef_buffer.fill(0)
d_input_pauli_buffer[:len(d_input_pauli)] = d_input_pauli
d_input_coef_buffer[:len(d_input_coef)] = d_input_coef

# Recreate expansions
cupp.destroy_pauli_expansion(input_expansion)
cupp.destroy_pauli_expansion(output_expansion)
input_expansion, output_expansion = create_expansions()

current_input = input_expansion
current_output = output_expansion

start_time = time.time()
term_history_no_trunc = []
max_terms_no_trunc = 0

print(f"\nApplying {len(gate_ops)} gates in reverse order WITHOUT truncation...\n")

for gate_idx, gate in enumerate(reversed(gate_ops)):
    gate_number = len(gate_ops) - gate_idx

    # Get current term count
    num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    term_history_no_trunc.append(num_terms)
    max_terms_no_trunc = max(max_terms_no_trunc, num_terms)

    # Create view
    input_view = cupp.pauli_expansion_get_contiguous_range(
        handle, current_input, 0, num_terms
    )

    # Apply gate WITHOUT truncation
    cupp.pauli_expansion_view_compute_operator_application(
        handle,
        input_view,
        current_output,
        gate,
        1,      # adjoint=1
        1,      # makeSorted=True
        0,      # keepDuplicates=False
        0,      # num_truncation_strategies=0 (NO TRUNCATION)
        None,   # truncation_strategies=None
        workspace
    )

    # Destroy view
    cupp.destroy_pauli_expansion_view(input_view)

    # Swap
    current_input, current_output = current_output, current_input

    new_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)

    if gate_idx % 10 == 0 or gate_idx == len(gate_ops) - 1:
        print(f"Gate {gate_number:3d}: {num_terms:7d} → {new_num_terms:7d} terms")

end_time = time.time()
final_num_terms_no_trunc = cupp.pauli_expansion_get_num_terms(handle, current_input)

print(f"\n{'='*70}")
print(f"WITHOUT TRUNCATION Results:")
print(f"  Final terms: {final_num_terms_no_trunc}")
print(f"  Max terms during evolution: {max_terms_no_trunc}")
print(f"  Runtime: {end_time - start_time:.2f} sec")
print(f"{'='*70}\n")

PROPAGATION WITHOUT TRUNCATION (Reference/Baseline)

Applying 156 gates in reverse order WITHOUT truncation...

Gate 156:       1 →       1 terms
Gate 146:       1 →       1 terms
Gate 136:      18 →      27 terms
Gate 126:     729 →     729 terms
Gate 116:   19683 →   19683 terms
Gate 106:   45927 →   45927 terms
Gate  96:  215055 →  261225 terms


#ERROR: Exception: Invalid argument: #ERROR(InvalidArgument): Number of terms is too large for the XZ buffer!


cuPauliPropError: INVALID_VALUE (2): CUPAULIPROP_STATUS_INVALID_VALUE

In [ ]:
print("="*70)
print("PROPAGATION WITHOUT TRUNCATION (Reference/Baseline)")
print("="*70)

# Reset buffers and expansions
d_input_pauli_buffer.fill(0)
d_input_coef_buffer.fill(0)
d_output_pauli_buffer.fill(0)
d_output_coef_buffer.fill(0)
d_input_pauli_buffer[:len(d_input_pauli)] = d_input_pauli
d_input_coef_buffer[:len(d_input_coef)] = d_input_coef

# Recreate expansions
cupp.destroy_pauli_expansion(input_expansion)
cupp.destroy_pauli_expansion(output_expansion)
input_expansion, output_expansion = create_expansions()

current_input = input_expansion
current_output = output_expansion

start_time = time.time()
term_history_no_trunc = []
max_terms_no_trunc = 0

print(f"\nApplying {len(gate_ops)} gates in reverse order WITHOUT truncation...\n")

for gate_idx, gate in enumerate(reversed(gate_ops)):
    gate_number = len(gate_ops) - gate_idx

    # Get current term count
    num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    term_history_no_trunc.append(num_terms)
    max_terms_no_trunc = max(max_terms_no_trunc, num_terms)

    # Create view
    input_view = cupp.pauli_expansion_get_contiguous_range(
        handle, current_input, 0, num_terms
    )

    # Apply gate WITHOUT truncation
    cupp.pauli_expansion_view_compute_operator_application(
        handle,
        input_view,
        current_output,
        gate,
        1,      # adjoint=1
        1,      # makeSorted=True
        0,      # keepDuplicates=False
        0,      # num_truncation_strategies=0 (NO TRUNCATION)
        None,   # truncation_strategies=None
        workspace
    )

    # Destroy view
    cupp.destroy_pauli_expansion_view(input_view)

    # Swap
    current_input, current_output = current_output, current_input

    new_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)

    if gate_idx % 10 == 0 or gate_idx == len(gate_ops) - 1:
        print(f"Gate {gate_number:3d}: {num_terms:7d} → {new_num_terms:7d} terms")

end_time = time.time()
final_num_terms_no_trunc = cupp.pauli_expansion_get_num_terms(handle, current_input)

print(f"\n{'='*70}")
print(f"WITHOUT TRUNCATION Results:")
print(f"  Final terms: {final_num_terms_no_trunc}")
print(f"  Max terms during evolution: {max_terms_no_trunc}")
print(f"  Runtime: {end_time - start_time:.2f} sec")
print(f"{'='*70}\n")

In [ ]:
# Compute expectation value without truncation
final_view_no_trunc = cupp.pauli_expansion_get_contiguous_range(
    handle, current_input, 0, final_num_terms_no_trunc
)

result_no_trunc = np.array([0.0], dtype=np.float64)
cupp.pauli_expansion_view_compute_trace_with_zero_state(
    handle,
    final_view_no_trunc,
    result_no_trunc.ctypes.data,
    workspace
)

expectation_no_trunc = result_no_trunc[0]
print(f"Expectation value (NO truncation): {expectation_no_trunc:.8f}")

# Store final state for debugging
if current_input == input_expansion:
    buffer_pauli_no_trunc = d_input_pauli_buffer
    buffer_coef_no_trunc = d_input_coef_buffer
else:
    buffer_pauli_no_trunc = d_output_pauli_buffer
    buffer_coef_no_trunc = d_output_coef_buffer

pauli_data_no_trunc = buffer_pauli_no_trunc[:final_num_terms_no_trunc*2*num_packed_ints].get()
coef_data_no_trunc = buffer_coef_no_trunc[:final_num_terms_no_trunc].get()

print(f"\nFinal Pauli Expansion (first 10 terms):")
for j in range(min(10, final_num_terms_no_trunc)):
    x_mask = pauli_data_no_trunc[2*j]
    z_mask = pauli_data_no_trunc[2*j + 1]
    coef = coef_data_no_trunc[j]
    print(f"  Term {j}: X={x_mask:010b} Z={z_mask:010b} coef={coef:+.6e}")

Expectation value (NO truncation): 0.09795688

Final Pauli Expansion (first 10 terms):
  Term 0: X=0000000001 Z=0000000000 coef=-2.535503e-02
  Term 1: X=0000000010 Z=0000000000 coef=-9.552084e-03
  Term 2: X=0000000011 Z=0000000000 coef=-5.147205e-03
  Term 3: X=0000000100 Z=0000000000 coef=-2.055933e-03
  Term 4: X=0000000101 Z=0000000000 coef=-1.714542e-02
  Term 5: X=0000000110 Z=0000000000 coef=+1.138128e-02
  Term 6: X=0000000111 Z=0000000000 coef=+1.031460e-02
  Term 7: X=0000001000 Z=0000000000 coef=-6.889691e-04
  Term 8: X=0000001001 Z=0000000000 coef=+5.930481e-03
  Term 9: X=0000001010 Z=0000000000 coef=+1.091152e-02


---
## Section 8: Comparison and Analysis

In [12]:
import cudaq
from cudaq import spin
@cudaq.kernel
def truth():    
    qv = cudaq.qvector(num_qubits)
    for l in range(NUM_LAYERS):
        for q in range(num_qubits):
            rx(theta, qv[q])
            ry(theta, qv[q])
            rz(theta, qv[q])
        for q in range(num_qubits - 1):
            x.ctrl(qv[q], qv[q+1])
gt = cudaq.observe(truth, spin.z(7)).expectation()

print("\n" + "="*70)
print("COMPARISON: WITH vs WITHOUT TRUNCATION")
print("="*70)

print(f"\nFinal Term Counts:")
print(f"  WITH truncation:    {final_num_terms_with_trunc:10d} terms")
# print(f"  WITHOUT truncation: {final_num_terms_no_trunc:10d} terms")
# term_reduction = 100 * (1 - final_num_terms_with_trunc / final_num_terms_no_trunc) if final_num_terms_no_trunc > 0 else 0
# print(f"  Reduction:          {term_reduction:10.1f}%")

print(f"\nMax Terms During Evolution:")
print(f"  WITH truncation:    {max_terms_with_trunc:10d} terms")
# print(f"  WITHOUT truncation: {max_terms_no_trunc:10d} terms")
# max_reduction = 100 * (1 - max_terms_with_trunc / max_terms_no_trunc) if max_terms_no_trunc > 0 else 0
# print(f"  Reduction:          {max_reduction:10.1f}%")

print(f"\nExpectation Values:")
print(f"  Gorund Truth Value: {gt:.8f}")
print(f"  WITH truncation:    {expectation_with_trunc:.8f}")
# print(f"  WITHOUT truncation: {expectation_no_trunc:.8f}")
abs_diff_trunc = abs(gt - expectation_with_trunc)
# abs_diff_no_trunc = abs(gt - expectation_no_trunc)
print(f"  Absolute diff (WITH truncation):      {abs_diff_trunc:.2e}")
# print(f"  Absolute diff (WITHOUT truncation):      {abs_diff_no_trunc:.2e}")

print(f"\nMemory Savings:")
# Rough estimate: each term uses (2*num_packed_ints*8 + 8) bytes
bytes_per_term = 2 * num_packed_ints * 8 + 8
mem_with_trunc = final_num_terms_with_trunc * bytes_per_term / 1024**2
# mem_no_trunc = final_num_terms_no_trunc * bytes_per_term / 1024**2
print(f"  WITH truncation:    {mem_with_trunc:.1f} MB")
# print(f"  WITHOUT truncation: {mem_no_trunc:.1f} MB")
# print(f"  Saved:              {mem_no_trunc - mem_with_trunc:.1f} MB ({100*(mem_no_trunc - mem_with_trunc)/mem_no_trunc:.1f}%)")

print(f"\n{'='*70}\n")


COMPARISON: WITH vs WITHOUT TRUNCATION

Final Term Counts:
  WITH truncation:         95652 terms

Max Terms During Evolution:
  WITH truncation:        290578 terms

Expectation Values:
  Gorund Truth Value: 0.04729039
  WITH truncation:    0.02130400
  Absolute diff (WITH truncation):      2.60e-02

Memory Savings:
  WITH truncation:    2.2 MB




In [ ]:
# Visualization of term count evolution
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Term count over gates
gates_range = list(range(1, len(term_history_with_trunc) + 1))
ax1.plot(gates_range, term_history_with_trunc, 'b-o', label='WITH truncation', linewidth=2, markersize=4, alpha=0.7)
ax1.plot(gates_range, term_history_no_trunc, 'r-s', label='WITHOUT truncation', linewidth=2, markersize=4, alpha=0.7)
ax1.axvline(x=len(gate_ops), color='gray', linestyle='--', alpha=0.5)
ax1.set_xlabel('Gate Index (reverse order)', fontsize=11)
ax1.set_ylabel('Number of Pauli Terms', fontsize=11)
ax1.set_title('Pauli Term Count Evolution', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Plot 2: Truncation effect at different gate counts
cumulative_gates = []
term_diff = []
for i, (t_with, t_no) in enumerate(zip(term_history_with_trunc, term_history_no_trunc)):
    cumulative_gates.append(i + 1)
    if t_no > 0:
        term_diff.append(100 * (t_no - t_with) / t_no)
    else:
        term_diff.append(0)

ax2.bar(cumulative_gates[::5], term_diff[::5], width=4, alpha=0.7, color='green')
ax2.set_xlabel('Gate Index (reverse order)', fontsize=11)
ax2.set_ylabel('Term Reduction (%)', fontsize=11)
ax2.set_title('Memory Savings from Truncation', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/tmp/truncation_analysis.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Visualization saved")

---
## Section 9: Resource Cleanup

In [ ]:
print("Cleaning up resources...\n")

# Destroy views
try:
    cupp.destroy_pauli_expansion_view(final_view_trunc)
    cupp.destroy_pauli_expansion_view(final_view_no_trunc)
    print("  ✓ Views destroyed")
except:
    pass

# Destroy gate operators
try:
    for gate in gate_ops:
        cupp.destroy_operator(gate)
    print(f"  ✓ {len(gate_ops)} gate operators destroyed")
except Exception as e:
    print(f"  Gate operators cleanup error (ignored): {e}")

# Destroy expansions
try:
    cupp.destroy_pauli_expansion(input_expansion)
    cupp.destroy_pauli_expansion(output_expansion)
    print("  ✓ Pauli expansions destroyed")
except Exception as e:
    print(f"  Expansions cleanup error (ignored): {e}")

# Destroy workspace
try:
    cupp.destroy_workspace_descriptor(workspace)
    print("  ✓ Workspace descriptor destroyed")
except Exception as e:
    print(f"  Workspace cleanup error (ignored): {e}")

# Free GPU memory
try:
    d_workspace.free()
    d_input_pauli_buffer = None
    d_input_coef_buffer = None
    d_output_pauli_buffer = None
    d_output_coef_buffer = None
    print("  ✓ GPU memory freed")
except Exception as e:
    print(f"  GPU memory cleanup error (ignored): {e}")

# Destroy handle
try:
    cupp.destroy(handle)
    print("  ✓ Handle destroyed")
except Exception as e:
    print(f"  Handle cleanup error (ignored): {e}")

print("\n✓ All resources cleaned up successfully!")

Cleaning up resources...

  ✓ Views destroyed
  ✓ 124 gate operators destroyed
  ✓ Pauli expansions destroyed
  ✓ Workspace descriptor destroyed
  GPU memory cleanup error (ignored): 'cupy.cuda.memory.MemoryPointer' object has no attribute 'free'
  ✓ Handle destroyed

✓ All resources cleaned up successfully!
